# Faster NPZ

- frame per profile
- all-in-one-Meta 분리

In [1]:
import os
import time
import pickle
import cv2 as cv
import numpy as np
import tensorflow as tf
import multiprocessing 
import concurrent.futures

from tqdm import tqdm
from pprint import pprint

In [3]:
"""
    Device Screen Camera Difference (cm)
"""
cam_screen_dist_meta = {
    'iphone 6s plus':   [ 23.54  ,8.66   ,68.36  ,121.54 ],
    'iphone 6s':        [ 18.61  ,8.04   ,58.49  ,104.05 ],
    'iphone 6 plus':    [ 23.54  ,8.65   ,68.36  ,121.54 ],
    'iphone 6':         [ 18.61  ,8.03   ,58.5   ,104.05 ],
    'iphone 5s':        [ 25.85  ,10.65  ,51.7   ,90.39  ],
    'iphone 5c':        [ 25.85  ,10.64  ,51.7   ,90.39  ],
    'iphone 5':         [ 25.85  ,10.65  ,51.7   ,90.39  ],
    'iphone 4s':        [ 14.96  ,9.78   ,49.92  ,74.88  ],
    'ipad mini':        [ 60.7   ,8.7    ,121.3  ,161.2  ],
    'ipad air 2':       [ 76.86  ,7.37   ,153.71 ,203.11 ],
    'ipad air':         [ 74.4   ,9.9    ,149    ,198.1  ],
    'ipad 4':           [ 74.5   ,10.5   ,149    ,198.1  ],
    'ipad 3':           [ 74.5   ,10.5   ,149    ,198.1  ],
    'ipad 2':           [ 74.5   ,10.5   ,149    ,198.1  ],
    'ipad pro':         [ 98.31  ,10.69  ,196.61 ,262.15 ]
}

device_data = {
    'iphone 4s': {
        'matrix': [606.59362793, 609.2008667, 236.86116028, 312.28497314],
        'distortion': [ 0.24675941, -0.65499198,  0.00301733, -0.00097767]
    },
    'iphone 5': {
        'matrix': [623.28759766, 626.64154053, 236.86317444, 316.909729],
        'distortion': [ 0.03760624, -0.043609, -0.00114902,  0.00269194]
    },
    'iphone 5c': {
        'matrix': [585.13171387, 588.14447021, 242.18914795, 321.20614624],
        'distortion': [ 0.01302955, -0.10349616, -0.0009803,  0.00301618]
    },
    'iphone 5s': {
        'matrix': [585.13171387, 588.14447021, 242.18914795, 321.20614624],
        'distortion': [ 0.01302955, -0.10349616, -0.0009803,  0.00301618]
    },
    'iphone 6': {
        'matrix': [592.50164795, 595.66986084, 236.12217712, 327.50753784],
        'distortion': [ 0.0822313, -0.18398251, -0.00631323, -0.00075782]
    },
    'iphone 6 plus': {
        'matrix': [592.50164795, 595.66986084, 236.12217712, 327.50753784],
        'distortion': [ 0.0822313, -0.18398251, -0.00631323, -0.00075782]
    },
    'iphone 6s': {
        'matrix': [592.50164795, 595.66986084, 236.12217712, 327.50753784],
        'distortion': [ 0.0822313, -0.18398251, -0.00631323, -0.00075782]
    },
    'iphone 6s plus': {
        'matrix': [592.50164795, 595.66986084, 236.12217712, 327.50753784],
        'distortion': [ 0.0822313, -0.18398251, -0.00631323, -0.00075782]
    },
    'iphone 7': {
        'matrix': [592.50164795, 595.66986084, 236.12217712, 327.50753784],
        'distortion': [ 0.0822313, -0.18398251, -0.00631323, -0.00075782]
    },
    'iphone 7 plus': {
        'matrix': [592.50164795, 595.66986084, 236.12217712, 327.50753784],
        'distortion': [ 0.0822313, -0.18398251, -0.00631323, -0.00075782]
    },
    'iphone 8': {
        'matrix': [580.34485, 581.34717, 239.41379, 319.58548],
        'distortion': [ 0.0822313, -0.18398251, -0.00631323, -0.00075782]
    },
    'iphone 8 plus': {
        'matrix': [580.34485, 581.34717, 239.41379, 319.58548],
        'distortion': [ 0.0822313, -0.18398251, -0.00631323, -0.00075782]
    },
    'iphone x': {
        'matrix': [592.16473, 593.1875, 242.00687, 320.23456],
        'distortion': [ 0.0822313, -0.18398251, -0.00631323, -0.00075782]
    },
    'ipad air': {
        'matrix': [578, 578, 240, 320],
        'distortion': [0.124, -0.214, 0, 0]
    },  
    'ipad air 2': {
        'matrix': [592.35223389, 595.9105835, 234.15885925, 313.48773193],
        'distortion': [ 1.93445340e-01, -5.54507077e-01,  6.13935478e-03,  3.40262457e-04]
    },
    'ipad 2': {
        'matrix': [621.54315186, 624.44012451, 233.66329956, 313.44387817],
        'distortion': [-0.0243901, -0.10230259, -0.00513017,  0.00057966]
    },
    'ipad mini': {
        'matrix': [623.28759766, 626.64154053, 236.86317444, 316.909729],
        'distortion': [ 0.03760624, -0.043609, -0.00114902,  0.00269194]
    },
}

In [4]:
def grep_recur(base_path, pattern="*.*"):
    from itertools import chain
    sub_greps = list(chain(*[grep_recur(dp, pattern) for dp in grep_dirs(base_path)]))
    return grep_files(base_path, pattern) + sub_greps

def grep_files(base_path, pattern="*.*"):
    import glob
    return glob.glob("{}/{}".format(base_path, pattern))

def grep_dirs(base_path):
    file_paths = [os.path.join(base_path, name) for name in os.listdir(base_path)]
    return [p for p in file_paths if os.path.isdir(p)]

def timeit(func):
    def timed(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print('%r  %2.2f s' % (func.__name__, (end - start)))
        return result
    return timed

def get_camera_matrix(fx, fy, cx, cy, skewed=1.0):
    return np.array([
        [fx, 0,     cx],
        [0, fy,     cy],
        [0,  0, skewed],
    ])

def get_camera_param_and_distortion(device_name):
    device_name = device_name.lower()
    
    if not(device_name in device_data):
        return None, None
    
    if device_data[device_name] == -1:
        return None, None
    
    di = device_data[device_name]
    camera_distortion = np.hstack((di['distortion'], 0))
    camera_matrix = get_camera_matrix(*di['matrix'])
    
    return camera_matrix, camera_distortion

def rect_to_tl_br(rect):
    h, w, y, x = rect['h'], rect['w'], rect['y'], rect['x']
    h, w, y, x = round(h), round(w), round(y), round(x)
    h, w, y, x = max(0, h), max(0, w), max(0, y), max(0, x)  # Profile 33번 음수 버그 있음
    return y, x, y+h, x+w

def get_eye_corner(face_rect, left_eye_rect, right_eye_rect, image_hw):
    # h, w = image_hw
    top, left, bottom, right = rect_to_tl_br(face_rect)
    ltop, lleft, lbottom, lright = rect_to_tl_br(left_eye_rect)
    rtop, rleft, rbottom, rright = rect_to_tl_br(right_eye_rect)

    # convert to frame base
    ltop, lleft, lbottom, lright = top+ltop, left+lleft, top+lbottom, left+lright
    rtop, rleft, rbottom, rright = top+rtop, left+rleft, top+rbottom, left+rright

    # [ x x x x y y y y ]
    return [ltop, lbottom, rtop, rbottom, lleft, lright, rleft, rright]

def get_eye_left_right_patch(frame, face_rect, left_rect, right_rect):
    top, left, bottom, right = rect_to_tl_br(face_rect)
    ltop, lleft, lbottom, lright = rect_to_tl_br(left_rect)
    rtop, rleft, rbottom, rright = rect_to_tl_br(right_rect)
    le_img = frame[top+ltop:top+lbottom, left+lleft:left+lright, :]
    re_img = frame[top+rtop:top+rbottom, left+rleft:left+rright, :]

    return le_img, re_img

def eye_corner_pos_to_uv(eye_corner, camera_mat):
    # eye corner landmark [ x1 x2 x3 x4 y1 y2 y3 y4 ]
    ec, cm = eye_corner, camera_mat
    fx, fy, cx, cy = cm[0, 0], cm[1, 1], cm[0, 2], cm[1, 2]
    xs, ys = ec[0:4], ec[4:]
    xs = (xs - cx) / fx    
    ys = (ys - cy) / fy    
    return np.hstack([xs, ys])

def preprocess_image(image, resize_wh):
    image = cv.resize(image, resize_wh, interpolation=cv.INTER_AREA)
    ycrcb = cv.cvtColor(image, cv.COLOR_RGB2YCrCb)
    ycrcb[:, :, 0] = cv.equalizeHist(ycrcb[:, :, 0])
    image = cv.cvtColor(ycrcb, cv.COLOR_YCrCb2RGB)
    image = 2.0 * image / 255.0 - 1
    # image = np.rollaxis(image, 2, 0)
    return image

# 데이터셋 위치

In [5]:
npz_root_path = "/home/elvin/banner/mnt/ssd3/everyone-npz/"
npz_out_path = "/home/elvin/banner/mnt/ssd3/ps/"
npz_paths = grep_recur(npz_root_path)

In [6]:
def summary_profile(npz):
    s, m = npz['summary'].tolist(), npz['meta']
    pid = s['profile_id']
    device = s['device']
    num_frames = int(s['num_frames'])
    
    print("[{}] {:20s} items: {:05d}".format(pid, device, num_frames))
    return pid

# 프레임 랜덤 엑세스 측정

In [ ]:
# sample_npy_path = "/home/elvin/banner/mnt/ssd3/ps/ps-01819.npy"
# frames = np.load(sample_npy_path)

# @timeit
# def random_access(frames):
#     ridx = np.random.randint(frames.shape[0], size=1)[0]
#     print("[{}]: len: {}".format(ridx, len(frames[ridx])))
    
# for i in range(10):
#     random_access(frames)

# 프레임 정제

In [ ]:
def write_frames(npz):
    frames = npz['frames']
    out_path = os.path.join(npz_out_path, "ps-{:05d}.npy".format(pid))
    np.save(out_path, frames)

In [ ]:
for p in npz_paths: 
    npz = np.load(p, allow_pickle=True)
    pid = int(summary_profile(npz))
    write_frames(npz)


# with concurrent.futures.ThreadPoolExecutor(max_workers=8) as e:
#     for p in npz_paths: 
#         npz = np.load(p, allow_pickle=True)
#         pid = int(summary_profile(npz))
        
#         try:
#             e.submit(write_frames, npz)
#         except Exception as ee:
#             print(ee)

# 메타 정제

In [7]:
def merge_metas(npz, merge_dict):
    s = npz['summary'].tolist()
    metas = npz['meta']
    device = s['device'].lower()
    pid = s['profile_id']
    
    csd = cam_screen_dist_meta[device]
    cam_param, distort = get_camera_param_and_distortion(device)
    
    for m in metas:
        m['cam_screen_dist'] = csd
        m['distortion_coefficient'] = distort
        m['cam_param'] = cam_param
        m['device'] = device
    
    merge_dict[pid] = metas

In [8]:
# manager = multiprocessing.Manager()
# merge_dict = manager.dict()   
# with concurrent.futures.ThreadPoolExecutor(max_workers=16) as e:

merging = {}
for p in npz_paths: 
    npz = np.load(p, allow_pickle=True)
    merge_metas(npz, merging)

In [9]:
out_path = os.path.join(npz_out_path, "meta.pkl")
with open(out_path, 'wb') as f: 
    pickle.dump(merging, f)

# 도메인 엔티티 표현

In [ ]:
class MetaItem:
    def __init__():
        self.pid = None
        self.device = "Unknown"
        
        'face_valid': True, 
        'face_grid_valid': True, 
        'left_eye_valid': True, 
        'right_eye_valid': True, 
            
        'face_rect': {'h': 343.669759177, 'w': 343.678719177, 'x': 38.1526404115, 'y': 230.037120412},
        'face_grid_rect': {'h': 13, 'w': 13, 'x': 6, 'y': 10}, 
        'left_eye_rect': {'h': 103.103613281, 'w': 103.103613281, 'x': 166.684172745, 'y': 79.037153815}, 
        'right_eye_rect': {'h': 103.103613281, 'w': 103.103613281, 'x': 32.6494754792, 'y': 89.3475151431}, 
        'target_id': 0, 
        'target_pts': {'x': 160, 'y': 284},
        'target_dist': {'x': 1.064, 'y': -6.0055}, 
        'target_time': 0.205642, 
        'screen_hw': {'h': 568, 'w': 320}, 
        'orientation': 1,
        
        'is_pad': False,
        'is_small_profile': False,
        